# TP 3: Chatbot à Règles, Intention + Mémoire de Contexte

## Objectifs du TP
- Implémenter un agent symbolique (sans apprentissage automatique)
- Détecter des intentions à partir de mots-clés
- Gérer une mémoire de contexte (dernier sujet abordé)
- Concevoir des modules distincts : détection, mémoire, génération de réponses
- Comprendre les limitations des systèmes à règles

In [20]:
import regex, random, datetime

## Partie I: Intention et Mots-Clés

Dans cette première partie, vous allez définir les intentions que votre chatbot peut reconnaître et les mots-clés associés à chaque intention.

**Q 1**: Définition du dictionnaire INTENTS

In [21]:
INTENTS = {
    "salutation": ["bonjour","salut","coucou","bonsoir","hey"],
    "meteo": ["meteo","temps","chaud","froid","pluie","soleil","automne","ete","printemps","hiver"],
    "heure": ["matin","soir","après-midi","heure","minute","seconde","temps", "aujourdhui", "demain", "hier"],
    "identite": ["qui","nom","objectif","but","pourquoi"],
    "aurevoir": ["bye","au revoir","ciao","à plus","à bientot","à demain"],
}

## Partie II: Normalisation du Texte
La normalisation du texte est une étape cruciale pour améliorer la robustesse de la détection  d'intentions. Elle permet de traiter uniformément les variations d'écriture. 


**Q 2**: Implémentation de la fonction normalize

In [22]:
accent_dict = {
    "a": ["à","â"],
    "e": ["é","è","ê","ë"],
    "u": ["ù","û","ü"],
    "o": ["ô"],
    "i": ["ï"],
    "c": ["ç"]
}

accent_map = {ch: base for base, lst in accent_dict.items() for ch in lst}

def normalize(text):

    text = str(text).lower()
    if accent_map:
        pattern = "[" + "".join(regex.escape(c) for c in accent_map.keys()) + "]"
        text = regex.sub(pattern, lambda m: accent_map[m.group(0)], text)

    # remove punctuation/symbols, collapse whitespace, strip
    text = regex.sub(r"[\p{P}\p{S}]+", " ", text)
    text = regex.sub(r"\s+", " ", text).strip()

    return text

In [27]:
# Contenu de accent_map
accent_map

{'à': 'a',
 'â': 'a',
 'é': 'e',
 'è': 'e',
 'ê': 'e',
 'ë': 'e',
 'ù': 'u',
 'û': 'u',
 'ü': 'u',
 'ô': 'o',
 'ï': 'i',
 'ç': 'c'}

## Partie 3: Détection d'Intention
Maintenant que vous avez défini les intentions et la normalisation, vous allez implémenter la  fonction qui détecte l'intention d'un message utilisateur. 


**Q 3**: Implémentation de detect_intent

In [23]:
def detect_intent(message):
    new_intent_table = {word:intent for intent, words in INTENTS.items() for word in words}
    message = normalize(message)
    for word in message.split(sep=" "):
        if word in new_intent_table:
            return new_intent_table[word]
        else:
            continue
    return "inconnu"
    

In [26]:
# new_intent_table
new_intent_table = {word:intent for intent, words in INTENTS.items() for word in words }
(new_intent_table)

{'bonjour': 'salutation',
 'salut': 'salutation',
 'coucou': 'salutation',
 'bonsoir': 'salutation',
 'hey': 'salutation',
 'meteo': 'meteo',
 'temps': 'heure',
 'chaud': 'meteo',
 'froid': 'meteo',
 'pluie': 'meteo',
 'soleil': 'meteo',
 'automne': 'meteo',
 'ete': 'meteo',
 'printemps': 'meteo',
 'hiver': 'meteo',
 'matin': 'heure',
 'soir': 'heure',
 'après-midi': 'heure',
 'heure': 'heure',
 'minute': 'heure',
 'seconde': 'heure',
 'aujourdhui': 'heure',
 'demain': 'heure',
 'hier': 'heure',
 'qui': 'identite',
 'nom': 'identite',
 'objectif': 'identite',
 'but': 'identite',
 'pourquoi': 'identite',
 'bye': 'aurevoir',
 'au revoir': 'aurevoir',
 'ciao': 'aurevoir',
 'à plus': 'aurevoir',
 'à bientot': 'aurevoir',
 'à demain': 'aurevoir'}

## Partie IV: Génération de Réponses + Mémoire
C'est la partie la plus importante du TP ! Vous allez implémenter la fonction qui génère les réponses en fonction de l'intention détectée et qui gère la mémoire de contexte. 

**Q 4**: Implémentation de respond avec mémoire

In [37]:
time_ids = INTENTS["heure"]
time_dict = {time:time for time in time_ids}
villes = {
    "casablanca": "casablanca",
    "rabat": "rabat",
    "fes": "fes",
    "marrakech": "marrakech",
    "agadir": "agadir",
    "tanger": "tanger"
}

FAKE_WEATHER = {
    "casablanca": 23,
    "rabat": 20,
    "fes": 24,
    "marrakech": 28,
    "agadir": 22,
    "tanger": 19
    }

In [ ]:
def respond(intent, message, memory):

    message = normalize(message)


    match intent:
        case "salutation":
            memory = {"last_intent": intent,
                      "last_city": "casablanca"}
            
            return INTENTS[intent][random.randrange(0,5)]
        
        case "aurevoir":
            memory = {"last_intent": intent,
                      "last_city": "casablanca"}
            
            return INTENTS[intent][random.randrange(0,6)]
        
        case "heure":
            memory = {"last_intent": intent,
                      "last_city": "casablanca"}
            
            return datetime.now()
        
        case "identite":
            memory = {"last_intent": intent,
                      "last_city": "casablanca"}
            
            return "Bonjour, je suis votre Agent IA, capable de converser avec vous sans aucun soucis."
        
        case "meteo":
            ville = [villes[city] for city in message.split(sep=" ") if city in villes]
            if not ville:
                ville = "casablanca"
            else:
                ville = ville[0]
            memory = {"last_intent": intent,
                      "last_city": ville}

            return f"{ville} est à {FAKE_WEATHER[ville]} actuellement."
        
        case "inconnu":
            if memory["last_intent"] == "meteo":
                time_ids = INTENTS["heure"] 
            
                ville = [villes[city] for city in message.split(sep=" ") if city in villes]
                heure = [time_dict[heure] for heure in message.split(sep=" ") if heure in time_ids]

                if not ville:
                    ville = memory["last_city"]
                else:
                    ville = ville[0]
                    return f"{ville} est à {FAKE_WEATHER[ville]} actuellement."
            
                

## Partie V: Boucle de Dialogue Interactive 
Maintenant que toutes les fonctions sont implémentées, vous allez créer une boucle  interactive permettant de discuter avec le chatbot. 


**Q 5**: Implémentation de chat_loop

In [ ]:
def chat_loop():
    pass